In [2]:
import matplotlib.pyplot as plt
matplotlib.style.use('default')
%matplotlib qt
import mne

In [3]:
labels = ['L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'L10',
          'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10']
x_coords = [-72, -81, -99, -108, -109, -109, -108, -99, -81, -72,
            72, 81, 99, 108, 109, 109, 108, 99, 81, 72]
y_coords = [0.67778, 0.73333, 0.73333, 0.67778, 0.63889, 0.60556, 0.56667, 0.51111, 0.51111, 0.56667,
            0.67778, 0.73333, 0.73333, 0.67778, 0.63889, 0.60556, 0.56667, 0.51111, 0.51111, 0.56667]
lpa_x = (x_coords[0] + x_coords[9])/2
lpa_y = (y_coords[0] + y_coords[9])/2

rpa_x = (x_coords[10] + x_coords[19])/2
rpa_y = (y_coords[10] + y_coords[19])/2

naison_x = (x_coords[2] + x_coords[11]+9)/2
naison_y = (y_coords[2] + y_coords[11])/2
print("Naison coordinates: ", naison_x, naison_y)
plt.figure(figsize=(8, 6))
scatter = plt.scatter(x_coords, y_coords, c='blue', marker='o')
plt.scatter(lpa_x, lpa_y, c='red', marker='x', label='LPA')
plt.scatter(rpa_x, rpa_y, c='green', marker='x', label='RPA')
plt.scatter(naison_x, naison_y, c='purple', marker='^', label='Nasion')
plt.annotate('LPA', xy=(lpa_x, lpa_y), xytext=(0, 5),
             textcoords="offset points",
             fontsize=8, ha='center')
plt.annotate('RPA', xy=(rpa_x, rpa_y), xytext=(0, 5),
                textcoords="offset points",
                fontsize=8, ha='center')
plt.annotate('Nasion', xy=(naison_x, naison_y), xytext=(0, 5),
                textcoords="offset points",
                fontsize=8, ha='center')

# Adding labels to each point
for i, label in enumerate(labels):
    plt.annotate(label,  # The text to display
                 (x_coords[i], y_coords[i]),  # The position to place the text
                 textcoords="offset points",  # How to position the text
                 xytext=(0, 5),  # Offset text by 5 points above
                 ha='center',  # Horizontal alignment
                 fontsize=8)  # Font size

# Adding a legend, title and axes labels
plt.legend()
plt.title('Electrode Positions')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')

Naison coordinates:  -4.5 0.73333


Text(0, 0.5, 'Y Coordinate')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Labels
labels = ['L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'L10',
          'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10']

# 2D electrode positions
x_coords = np.array([-72, -81, -99, -108, -109, -109, -108, -99, -81, -72,
                     72, 81, 99, 108, 109, 109, 108, 99, 81, 72])
y_coords = np.array([0.67778, 0.73333, 0.73333, 0.67778, 0.63889, 0.60556, 0.56667, 0.51111, 0.51111, 0.56667,
                     0.67778, 0.73333, 0.73333, 0.67778, 0.63889, 0.60556, 0.56667, 0.51111, 0.51111, 0.56667])
elec_2d = np.column_stack([x_coords, y_coords])

# Real 3D fiducials (in meters)
lpa_3d = np.array([-0.08061612, -0.02908875, -0.04131077])
rpa_3d = np.array([0.08436285, -0.02850276, -0.04127743])
nasion_3d = np.array([0.00146763,  0.08506715, -0.03483611])
tgt_fids = np.vstack([lpa_3d, rpa_3d, nasion_3d])

# Known physical distance between L1 and L10 is 4 cm
l1_2d = elec_2d[0]
l10_2d = elec_2d[9]
euclidean_dist = np.linalg.norm(l1_2d - l10_2d)
real_world_dist = 0.04  # meters
scale = real_world_dist / euclidean_dist
elec_2d_scaled = elec_2d * scale  # Now in meters

# Compute fiducials in scaled space
lpa_2d = np.mean([elec_2d_scaled[0], elec_2d_scaled[9]], axis=0)
rpa_2d = np.mean([elec_2d_scaled[10], elec_2d_scaled[19]], axis=0)
nasion_2d = np.mean([elec_2d_scaled[2], elec_2d_scaled[12]], axis=0)
src_fids = np.vstack([lpa_2d, rpa_2d, nasion_2d])

# Fit 2D → 3D affine transform
reg = LinearRegression()
reg.fit(src_fids, tgt_fids)
A = reg.coef_
b = reg.intercept_
elec_3d = elec_2d_scaled @ A.T + b

# Plot the 3D electrode positions
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(elec_3d[:, 0], elec_3d[:, 1], elec_3d[:, 2], c='b')

for i, label in enumerate(labels):
    ax.text(elec_3d[i, 0], elec_3d[i, 1], elec_3d[i, 2], label)

ax.set_title("Transformed Electrode Positions (3D)")
ax.set_xlabel("X (m)")
ax.set_ylabel("Y (m)")
ax.set_zlabel("Z (m)")
plt.tight_layout()
plt.show()


In [ ]:

# Create montage from transformed 3D electrode positions
ch_pos = dict(zip(labels, elec_3d))
montage = mne.channels.make_dig_montage(ch_pos=ch_pos, coord_frame='head')

# Create a fake info object
info = mne.create_info(ch_names=labels, sfreq=1000., ch_types='eeg')
info.set_montage(montage)

# Plot in 2D
fig_2d = info.plot_sensors(kind='topomap', show_names=True)

# Plot in 3D
# fig_3d = mne.viz.plot_alignment(info, dig=True, show_axes=True)

# (fig_2d, fig_3d)


In [ ]:
montages = mne.channels.get_builtin_montages(descriptions=True)
easycap_montage = mne.channels.make_standard_montage(montages[1][0])
print(easycap_montage)
easycap_montage.get_positions()

# easycap_montage.plot(kind="topomap")